# Assignment 2 - Pandas Introduction
All questions are weighted the same in this assignment.
## Part 1
The following code loads the olympics dataset (olympics.csv), which was derrived from the Wikipedia entry on [All Time Olympic Games Medals](https://en.wikipedia.org/wiki/All-time_Olympic_Games_medal_table), and does some basic data cleaning. 

The columns are organized as # of Summer games, Summer medals, # of Winter games, Winter medals, total # number of games, total # of medals. Use this dataset to answer the questions below.

In [51]:
import pandas as pd

df = pd.read_csv('olympics.csv', index_col=0, skiprows=1)

for col in df.columns:
    if col[:2]=='01':
        df.rename(columns={col:'Gold'+col[4:]}, inplace=True)
    if col[:2]=='02':
        df.rename(columns={col:'Silver'+col[4:]}, inplace=True)
    if col[:2]=='03':
        df.rename(columns={col:'Bronze'+col[4:]}, inplace=True)
    if col[:1]=='№':
        df.rename(columns={col:'#'+col[1:]}, inplace=True)

names_ids = df.index.str.split('\s\(') # split the index by '('

df.index = names_ids.str[0] # the [0] element is the country name (new index) 
df['ID'] = names_ids.str[1].str[:3] # the [1] element is the abbreviation or ID (take first 3 characters from that)

df = df.drop('Totals')
df.head()

,# Summer,Gold,Silver,Bronze,Total,# Winter,Gold.1,Silver.1,Bronze.1,Total.1,# Games,Gold.2,Silver.2,Bronze.2,Combined total,ID
Afghanistan,13,0,0,2,2,0,0,0,0,0,13,0,0,2,2,AFG
Algeria,12,5,2,8,15,3,0,0,0,0,15,5,2,8,15,ALG
Argentina,23,18,24,28,70,18,0,0,0,0,41,18,24,28,70,ARG
Armenia,5,1,2,9,12,6,0,0,0,0,11,1,2,9,12,ARM
Australasia,2,3,4,5,12,0,0,0,0,0,2,3,4,5,12,ANZ


### Question 0 (Example)

What is the first country in df?

*This function should return a Series.*

In [4]:
def answer_zero():
    return df.iloc[0]

answer_zero() 

# Summer           13
Gold                0
Silver              0
Bronze              2
Total               2
# Winter            0
Gold.1              0
Silver.1            0
Bronze.1            0
Total.1             0
# Games            13
Gold.2              0
Silver.2            0
Bronze.2            2
Combined total      2
ID                AFG
Name: Afghanistan, dtype: object

### Question 1
Which country has won the most gold medals in summer games?

*This function should return a single string value.*

In [11]:
def answer_one():
    
    return list(df[df["Gold"] == max(df["Gold"])].index)[0]

answer_one()

'United States'

### Question 2
Which country had the biggest difference between their summer and winter gold medal counts?

*This function should return a single string value.*

In [14]:
def answer_two():
    return df[(df['Gold'] - df['Gold.1']) == max(df['Gold'] - df['Gold.1'])]

answer_two()

,# Summer,Gold,Silver,Bronze,Total,# Winter,Gold.1,Silver.1,Bronze.1,Total.1,# Games,Gold.2,Silver.2,Bronze.2,Combined total,ID
United States,26,976,757,666,2399,22,96,102,84,282,48,1072,859,750,2681,USA


### Question 3
Which country has the biggest difference between their summer gold medal counts and winter gold medal counts relative to their total gold medal count? 

$$\frac{Summer~Gold - Winter~Gold}{Total~Gold}$$

Only include countries that have won at least 1 gold in both summer and winter.

*This function should return a single string value.*

In [46]:
def answer_three():
    copy_df = df[(df['Gold'] > 0) & (df['Gold.1'] > 0)].copy()
    total = copy_df['Gold'] + copy_df['Gold.1']
    diff = copy_df['Gold'] - copy_df['Gold.1']
    return copy_df[((copy_df['Gold'] - copy_df['Gold.1']) / copy_df['Gold.2']) == max(diff / total)]

answer_three()

,# Summer,Gold,Silver,Bronze,Total,# Winter,Gold.1,Silver.1,Bronze.1,Total.1,# Games,Gold.2,Silver.2,Bronze.2,Combined total,ID
Bulgaria,19,51,85,78,214,19,1,2,3,6,38,52,87,81,220,BUL


### Question 4
Write a function that creates a Series called "Points" which is a weighted value where each gold medal (`Gold.2`) counts for 3 points, silver medals (`Silver.2`) for 2 points, and bronze medals (`Bronze.2`) for 1 point. The function should return only the column (a Series object) which you created, with the country names as indices.

*This function should return a Series named `Points` of length 146*

In [50]:
def answer_four():
    ans = df['Gold.2'] * 3 + df['Silver.2'] * 2 + df['Bronze.2']
    return ans

answer_four()
df.head()

,# Summer,Gold,Silver,Bronze,Total,# Winter,Gold.1,Silver.1,Bronze.1,Total.1,# Games,Gold.2,Silver.2,Bronze.2,Combined total,ID
Afghanistan,13,0,0,2,2,0,0,0,0,0,13,0,0,2,2,AFG
Algeria,12,5,2,8,15,3,0,0,0,0,15,5,2,8,15,ALG
Argentina,23,18,24,28,70,18,0,0,0,0,41,18,24,28,70,ARG
Armenia,5,1,2,9,12,6,0,0,0,0,11,1,2,9,12,ARM
Australasia,2,3,4,5,12,0,0,0,0,0,2,3,4,5,12,ANZ


## Part 2
For the next set of questions, we will be using census data from the [United States Census Bureau](http://www.census.gov). Counties are political and geographic subdivisions of states in the United States. This dataset contains population data for counties and states in the US from 2010 to 2015. [See this document](https://www2.census.gov/programs-surveys/popest/technical-documentation/file-layouts/2010-2015/co-est2015-alldata.pdf) for a description of the variable names.

The census dataset (census.csv) should be loaded as census_df. Answer questions using this as appropriate.

### Question 5
Which state has the most counties in it? (hint: consider the sumlevel key carefully! You'll need this for future questions too...)

*This function should return a single string value.*

In [81]:
census_df = pd.read_csv('census.csv')


len(census_df[(census_df["STATE"] == 4) & (census_df["SUMLEV"] == 50)] )
census_df.head()

,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,...,RDOMESTICMIG2011,RDOMESTICMIG2012,RDOMESTICMIG2013,RDOMESTICMIG2014,RDOMESTICMIG2015,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015
0,40,3,6,1,0,Alabama,Alabama,4779736,4780127,4785161,...,0.002295,-0.193196,0.381066,0.582002,-0.467369,1.030015,0.826644,1.383282,1.724718,0.712594
1,50,3,6,1,1,Alabama,Autauga County,54571,54571,54660,...,7.242091,-2.915927,-3.012349,2.265971,-2.530799,7.606016,-2.626146,-2.722002,2.592270,-2.187333
2,50,3,6,1,3,Alabama,Baldwin County,182265,182265,183193,...,14.832960,17.647293,21.845705,19.243287,17.197872,15.844176,18.559627,22.727626,20.317142,18.293499
3,50,3,6,1,5,Alabama,Barbour County,27457,27457,27341,...,-4.728132,-2.500690,-7.056824,-3.904217,-10.543299,-4.874741,-2.758113,-7.167664,-3.978583,-10.543299
4,50,3,6,1,7,Alabama,Bibb County,22915,22919,22861,...,-5.527043,-5.068871,-6.201001,-0.177537,0.177258,-5.088389,-4.363636,-5.403729,0.754533,1.107861


In [94]:
def answer_five():
    ans = ''
    max_ = 1
    
    for i in range(0, max(census_df['STATE'])):
        if len(census_df[(census_df["STATE"] == i) & (census_df["SUMLEV"] == 50)]) > max_:
            ans = (list(census_df[census_df["STATE"] == i]["STNAME"])[0])
            max_ = len(census_df[(census_df["STATE"] == i) & (census_df["SUMLEV"] == 50)])
    return ans

answer_five()

'Texas'

### Question 6
**Only looking at the three most populous counties for each state**, what are the three most populous states (in order of highest population to lowest population)? Use `CENSUS2010POP`.

*This function should return a list of string values.*

In [124]:
def answer_six():
    ans = {}
    max_ = 1
    
    for i in range(1, max(census_df['STATE'])):
        
        state = census_df[census_df['STATE'] == i].copy()
        if not len(state):
            continue
        state.sort_values('CENSUS2010POP', ascending=False, inplace=True)
        sum_ = sum(state.head(3)['CENSUS2010POP'])
        #print(state['STNAME'])
        #sum_ = state.iloc[0]['CENSUS2010POP'] + state.iloc[1]['CENSUS2010POP'] + state.iloc[2]['CENSUS2010POP']
        ans[list(state['STNAME'])[0]] = sum_
        
    
    return list(reversed(sorted(ans, key=ans.get)))

answer_six()

['California',
 'Texas',
 'New York',
 'Florida',
 'Illinois',
 'Pennsylvania',
 'Ohio',
 'Michigan',
 'Georgia',
 'North Carolina',
 'Arizona',
 'New Jersey',
 'Virginia',
 'Washington',
 'Massachusetts',
 'Tennessee',
 'Indiana',
 'Missouri',
 'Maryland',
 'Wisconsin',
 'Minnesota',
 'Colorado',
 'Alabama',
 'South Carolina',
 'Louisiana',
 'Connecticut',
 'Kentucky',
 'Oregon',
 'Oklahoma',
 'Nevada',
 'Utah',
 'Kansas',
 'Iowa',
 'Arkansas',
 'Mississippi',
 'New Mexico',
 'Nebraska',
 'Hawaii',
 'West Virginia',
 'Idaho',
 'New Hampshire',
 'Rhode Island',
 'Maine',
 'Delaware',
 'Montana',
 'District of Columbia',
 'Alaska',
 'South Dakota',
 'North Dakota',
 'Vermont']

### Question 7
Which county has had the largest absolute change in population within the period 2010-2015? (Hint: population values are stored in columns POPESTIMATE2010 through POPESTIMATE2015, you need to consider all six columns.)

e.g. If County Population in the 5 year period is 100, 120, 80, 105, 100, 130, then its largest change in the period would be |130-80| = 50.

*This function should return a single string value.*

In [153]:
def answer_seven():
    keys = ['POPESTIMATE201' + str(i) for i in range(0, 6)]
    buf = census_df[keys].copy()
    
    max_ = 0
    ans = ''
    for idx, row in buf.iterrows():
        if max(row[keys]) - min(row[keys]) > max_:
            max_ = max(row[keys]) - min(row[keys])
            ans = census_df.iloc[idx]["CTYNAME"]
    return ans

answer_seven()

'Texas'

### Question 8
In this datafile, the United States is broken up into four regions using the "REGION" column. 

Create a query that finds the counties that belong to regions 1 or 2, whose name starts with 'Washington', and whose POPESTIMATE2015 was greater than their POPESTIMATE 2014.

*This function should return a 5x2 DataFrame with the columns = ['STNAME', 'CTYNAME'] and the same index ID as the census_df (sorted ascending by index).*

In [160]:
def answer_eight():
    return census_df[((census_df['REGION'] == 1) | (census_df['REGION'] == 2)) & (census_df['POPESTIMATE2015'] > census_df['POPESTIMATE2014']) & (census_df["CTYNAME"].str.startswith('Washington'))]

answer_eight()

,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,...,RDOMESTICMIG2011,RDOMESTICMIG2012,RDOMESTICMIG2013,RDOMESTICMIG2014,RDOMESTICMIG2015,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015
896,50,2,4,19,183,Iowa,Washington County,21704,21704,21697,...,5.743693,2.468684,-0.364282,0.272171,1.849596,5.881542,2.560117,-0.273212,0.408256,1.984933
1419,50,2,4,27,163,Minnesota,Washington County,238136,238134,238995,...,2.555118,3.260957,2.285952,3.072543,0.678755,4.178076,4.907905,4.009584,5.040262,2.615204
2345,50,1,2,42,125,Pennsylvania,Washington County,207820,207820,207877,...,3.683135,3.260434,1.219337,2.113561,1.656917,4.062988,3.750219,1.742595,2.762039,2.305276
2355,50,1,1,44,9,Rhode Island,Washington County,126979,127094,127085,...,-4.669304,-2.659574,1.551904,1.400112,0.506035,-3.533527,-1.788880,2.462459,2.428442,1.533918
3163,50,2,3,55,131,Wisconsin,Washington County,131887,131885,131967,...,-0.794876,0.785279,-2.215465,1.601149,-0.434498,-0.431504,1.162817,-1.763330,2.104796,0.059931
